In [1]:
%load_ext autoreload
%autoreload 2

from coeditor.common import *
import os

os.chdir(proj_root())

In [3]:
from coeditor.retrieval_model import RetrievalEditorModel
from coeditor.api import EditPredictionService, BasicQueryEditEncoder, BatchArgs

model_path = get_model_dir(False) / "coeditor-large-query-basic/checkpoint-221507"
model = RetrievalEditorModel.load(model_path)
model.to("cuda:1")
service = EditPredictionService(
    proj_root(),
    model,
    encoder=BasicQueryEditEncoder(max_ref_tks=512),
    batch_args=BatchArgs(
        max_ref_tks=50 * 512, max_ref_dropout=0.0, shuffle_extra_ids=False
    ),
)


/home/jiayi/Projects/SPOT/src/coeditor/retrieval_model.py:255: UserWarning: No extra args found, using default model setting.
  warnings.warn("No extra args found, using default model setting.")


In [10]:
service.suggest_edit(Path("src/coeditor/api.py"), 240)
# service.suggest_edit(Path("src/spot/utils.py"), (218, 0))

Element span: CodeRange(start=CodePosition(line=236, column=0), end=CodePosition(line=250, column=15))
num batches: 1
Batch stats: {'mean': '1.0', 'median': '1.0', 'min': '1.0', 'max': '1.0'}
len(input_tks) = 216
len(references) = 25
========== Predicted code change ==========
* Modified: 
    # path: coeditor.api/get_elem_by_line
    def get_elem_by_line(module: PythonModule, line: int) -> PythonElem | None:
        # def to_tuple(pos: CodePosition):
        #     return pos.line, pos.column

        # def in_span(loc: CodePosition, span: CodeRange):
        #     return to_tuple(span.start) <= to_tuple(loc) <= to_tuple(span.end)
        def in_span(line: int, span: CodeRange):
            return span.start.line <= line <= span.end.line

        for e in module.all_elements():
            span = module.location_map[e.tree]
            if in_span(line, span):
                print("Element span:", span)
                return e
        return None


In [11]:
service.tlogger.as_dataframe()


,name,count,avg_time,total_time
3,run model,4,0.630360,2.521439
1,construct project edit,4,0.245847,0.983387
0,get target element,5,0.163506,0.817528
2,encode edits,4,0.095208,0.380831
